### Урок 6. Взаимосвязь величин. Параметрические и непараметрические показатели корреляции. Корреляционный анализ.#

### -- Автор: Шенк Евгений Станиславович

In [1]:
import warnings

import math
import numpy as np
import pandas as pd
from scipy import stats
from itertools import combinations

warnings.filterwarnings('ignore')

### Задание 1.

Даны значения величины заработной платы заемщиков банка (salary) и значения их поведенческого кредитного скоринга (scoring):

salary = [35, 45, 190, 200, 40, 70, 54, 150, 120, 110]

scoring = [401, 574, 874, 919, 459, 739, 653, 902, 746, 832]

Используя только встроенные питоновские функции и структуры данных (т.е. без библиотек numpy, pandas и др.) найдите:
1. ковариацию (смещённую и несмещённую) этих двух величин,
2. коэффициент корреляции Пирсона,
3. коэффициент корреляции Кендалла.

Можно затем посчитать те же значения с использованием библиотек, чтобы проверить себя.

In [2]:
salary = [35, 45, 190, 200, 40, 70, 54, 150, 120, 110]
scoring = [401, 574, 874, 919, 459, 739, 653, 902, 746, 832]

1. ковариацию (смещённую и несмещённую) этих двух величин

Формула ковариации случайных величин $X$ и $Y$:
$$\operatorname{cov}(X, Y) = M\left( (X - M(X)) (Y - M(Y)) \right).$$

In [3]:
salary_mean = sum(salary) / len(salary)
scoring_mean = sum(scoring) / len(scoring)
a = [(x - salary_mean) for x in salary]
b = [(x - scoring_mean) for x in scoring]
cov_biased = sum([a * b for a,b in zip(a, b)]) / len([a * b for a,b in zip(a, b)])
cov_biased

9157.84

Несмещённая оценка ковариации:
$$\sigma_{xy} = \dfrac{1}{n - 1} \displaystyle\sum_{i = 1}^n (x_i - \overline{x}) \cdot (y_i - \overline{y})$$

In [4]:
cov_unbiased = sum([a * b for a,b in zip(a, b)]) / (len([a * b for a,b in zip(a, b)]) - 1)
cov_unbiased

10175.377777777778

2. коэффициент корреляции Пирсона

$$r_{XY} = \dfrac{\operatorname{cov}(X, Y)}{\sigma_X \cdot \sigma_Y}.$$

In [5]:
salary_std = math.sqrt((sum([abs(x)**2 for x in a]) / (len(a)-1)))
scoring_std = math.sqrt((sum([abs(x)**2 for x in b]) / (len(b)-1)))
salary_std, scoring_std 

(62.31318747531162, 183.99544076477068)

In [6]:
r = cov_unbiased / (salary_std * scoring_std)
r

0.8874900920739162

3. коэффициент корреляции Кендалла.

Т.к. в выборках отсутствуют повторяющиеся значения, то сразу считаем согласованные и несогл. пары

In [7]:
def is_concordant(pair1: tuple, pair2: tuple) -> bool:
    """Являются ли две пары согласованными.
    """
    
    return (pair1[0] - pair2[0]) * (pair1[1] - pair2[1]) > 0

In [8]:
P = 0
Q = 0

for pair1, pair2 in combinations(zip(salary, scoring), r=2):
    if is_concordant(pair1, pair2):
        P += 1
    else:
        Q += 1
        
P, Q

(43, 2)

In [9]:
tau = (P - Q) / (P + Q)
tau

0.9111111111111111

## Проверка

In [10]:
cov = np.cov(salary, scoring)
cov

array([[ 3882.93333333, 10175.37777778],
       [10175.37777778, 33854.32222222]])

In [11]:
np.std(salary, ddof=1), np.std(scoring, ddof=1)

(62.31318747531162, 183.99544076477068)

In [12]:
corr = np.corrcoef(salary, scoring)
corr

array([[1.        , 0.88749009],
       [0.88749009, 1.        ]])

In [13]:
corr_K = pd.DataFrame({'salary':salary, 'scoring':scoring}).corr(method='kendall')
corr_K

,salary,scoring
salary,1.000000,0.911111
scoring,0.911111,1.000000


### Задание 2.

Измерены значения IQ выборки студентов, обучающихся в местных технических вузах:

131, 125, 115, 122, 131, 115, 107, 99, 125, 111

Известно, что в генеральной совокупности IQ распределен нормально. Найдите доверительный интервал для математического ожидания с надежностью 0.95.

In [14]:
iq = np.array([131, 125, 115, 122, 131, 115, 107, 99, 125, 111])
p = 0.95
alpha = 1 - p
n = len(iq)

In [15]:
mean = iq.mean()
mean

118.1

In [16]:
std = iq.std(ddof=1)
std

10.54566788359614

In [17]:
# т.к. IQ распределен нормально, а дисперсия не известна, то используем t-распределение
t = stats.t.ppf(1 - alpha / 2, df=n - 1)
t

2.2621571627409915

In [18]:
# Находим доверительный интервал для математического ожидания
(mean - t * std / np.sqrt(n), mean + t * std / np.sqrt(n))

(110.55608365158724, 125.64391634841274)

### Задание 3.

Известно, что рост футболистов в сборной распределён нормально с известной дисперсией 25. На выборке объёма 27 выборочное среднее составило 174.2. Найдите доверительный интервал для математического ожидания с надёжностью 0.95.

In [19]:
var = 25
std = np.sqrt(var)
n = 27
mean = 174.4
p = 0.95
alpha = 1 - p

In [20]:
# т.к. рост распределен нормально и известна дисперсия используем нормальное распределение
t = stats.norm.ppf(1 - alpha / 2)
t

1.959963984540054

In [21]:
# Находим доверительный интервал для математического ожидания
(mean - t * std / np.sqrt(n), mean + t * std / np.sqrt(n))

(172.51402377653972, 176.2859762234603)